# Implementation of: 
Differentially Private Federated Learning: A Client Level Perspective (https://arxiv.org/abs/1712.07557)

In [0]:
import os
def download_github_code():
    os.system("wget https://raw.githubusercontent.com/tensorflow/privacy/master/privacy/analysis/rdp_accountant.py")

download_github_code()

In [0]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np
from IPython.display import clear_output
from numpy import linalg as LA
from rdp_accountant import compute_rdp  # pylint: disable=g-import-not-at-top
from rdp_accountant import get_privacy_spent
import warnings
warnings.filterwarnings('ignore')


# The client class

The client class implements the update method. That is, for a given weights that the server sends, the client train those models in his data and then send the deltas and the L2 norm.

The constructor of the class needs: 



*   number: The number of this client. Every client get a number from 0 to num_clients (only needed for debug)
*   loader: The data that is private to this client
* state_dict: The weights for the first model
* batch_size: The batch size that is going to use the client training the local model
* epochs: The number of epochs
* lr: The learning rate



I'm returning the deltas in a dict with the same keys as the state_dict of the "parent" model.

* wt1: Dict with the same keys as state_dict with the deltas of all the weights
* S: Dict with the same keys as state_dict with the L2 norms of all deltas

In [0]:

class client():
  def __init__(self, number, loader, state_dict, batch_size = 32, epochs=2, lr=0.01):
    self.number = number
    self.model = t_model()
    self.model.load_state_dict(state_dict)
    self.criterion = nn.NLLLoss()
    self.optimizer = optim.SGD(self.model.parameters(), lr=lr)
    self.epochs = epochs
    self.device =  device =  torch.device("cuda:0""cuda:0" if torch.cuda.is_available() else "cpu")
    self.dataLoader = loader                                       
                                           
                                  
  #
  def update(self, state_dict):
    w0 = state_dict
    self.model.load_state_dict(state_dict)
    self.model.to(self.device)
    running_loss = 0
    accuracy = 0
    for e in range(self.epochs):
        # Model in training mode, dropout is on
        self.model.train()
        accuracy=0
        running_loss = 0
        for images, labels in self.dataLoader:            
            images, labels = images.to(self.device), labels.to(self.device)                       
            self.optimizer.zero_grad()            
            output = self.model.forward(images)
            loss = self.criterion(output, labels)
            loss.backward()
            self.optimizer.step()            
            running_loss += loss.item()        
    S ={} 
    wt1 = {}
    for key, value in w0.items():
      wt1[key] = self.model.state_dict()[key]  - value   
      S[key] = LA.norm(wt1[key].cpu(), 2)
    return wt1, S


# The server class

The server class implements the server that holds the central model. 

The server class needs:

* number of clients that is going to create
* p_budget: The delta bugdet that we have for the training rounds
* Epsilon: The desired epsilon for the Diff Privacy

In [0]:
#@title
class server():
  def __init__(self, number_clients, p_budget, epsilon, sigmat = 1.12):
    self.model = t_model()
    self.sigmat = sigmat   
    self.n_clients = number_clients
    self.samples = get_samples(self.n_clients)
    self.clients = list()
    for i in range(number_clients):
      loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=32, sampler=self.samples[i])
      self.clients.append(client(i, loader, self.model.state_dict()))
    self.p_budget = p_budget
    self.epsilon = epsilon
    self.testLoader = torch.utils.data.DataLoader(mnist_testset, batch_size=32)
    self.device = torch.device("cuda:0""cuda:0" if torch.cuda.is_available() else "cpu")
    self.orders = ([1.25, 1.5, 1.75, 2., 2.25, 2.5, 3., 3.5, 4., 4.5] +
            list(range(5, 64)) + [128, 256, 512])
    
    
  #Evaluates the accuracy of the current model with the test data.  
  def eval_acc(self):
    self.model.to(self.device)
    #print('Aqui voy!')
    running_loss = 0
    accuracy = 0
    self.model.eval()
    suma=0
    total = 0
    running_loss = 0
    for images, labels in self.testLoader:            
        images, labels = images.to(self.device), labels.to(self.device) 
        output = self.model.forward(images)             
        ps = torch.exp(output)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        total += equals.size(0)
        suma = suma + equals.sum().item()
    else:      
        print('Accuracy: ',suma/float(total))
   



 #Given:
 # mt: number of clients involved in this round. 
  #deltas: list of dicts with the deltas of every client
  #norms: list of dicts with the norms of every weights for every client
  #sigma: Sigma to calculate the StdDistribution of the GaussianNormalNoise
  #state_dicst: Dict with the current model weights 
  
# This functions apply noise to the given deltas. 

  
  def sanitaze(self,mt, deltas, norms, sigma, state_dict):    
    new_dict = {}
    for key, value in state_dict.items():
      S=[]
      for i in range(len(norms)):        
        S.append(norms[i][key])
      S_value = np.median(S)      
      wt = value
      prom = 1/float(mt)       
      suma = 0
      for i in range(len(deltas)):    
        clip = (max(1, float(norms[i][key]/S_value)))            
        suma = suma + ((deltas[i][key] / clip ))
      noise = np.random.normal(0, float(S_value * sigma), size = suma.shape)      
      suma = suma.cpu().numpy()
      suma = suma*prom
      noise = noise*prom
      suma = suma + noise  
      
      suma = torch.from_numpy(suma)
      suma = wt + suma.float()
      new_dict[key] = suma
    return new_dict
    
 
  '''
  Given the number of clients that we're going to train in each round:
  First computes the RDP Privacy Involved in doing a training on:
    q: Sampling ratio. The number of clients involved in one round/total of clients
    sigmat: The sigma of the Gaussian noise. For this implementation, sigma is the ratio between 
            StdDeviation and the Sensitivity of the function. In our case the sensitivity of 
            the function is S. That's why the GaussianNoise is S*sigmat
    i: Number of times that we have applied this Gaussian Mechanism
    orders: As of today, I don't have any idea of what this is 
    This returns the RDP Privacy of the set of operations.
  Then based on the list of RDP, the orders, and a target epsilon we get the delta_spent
  '''
  def server_exec(self,mt):    
    i=1
    while(True):
      clear_output()
      print('Comunication round: ', i)
      self.eval_acc()         
      rdp = compute_rdp(float(mt/len(self.clients)), self.sigmat, i, self.orders)
      _,delta_spent, opt_order = get_privacy_spent(self.orders, rdp, target_eps=self.epsilon)
      print('Delta spent: ', delta_spent)
      print('Delta budget: ', self.p_budget)    
      if self.p_budget < delta_spent:
        break
      Zt = np.random.choice(self.clients, mt)      
      deltas = []
      norms = []
      for client in Zt:
        #print(client.number)
        deltaW, normW = client.update(self.model.state_dict())        
        deltas.append(deltaW)
        norms.append(normW)
      #print('all updates')      
      self.model.to('cpu')
      new_state_dict = self.sanitaze(mt, deltas, norms, self.sigmat, self.model.state_dict())
      #print('sanitaze')
      self.model.load_state_dict(new_state_dict)
      i+=1

# The model class

This is the class where we define the model of our setup

In [0]:
class t_model(nn.Module):
    def __init__(self):
        super(t_model, self).__init__()
        self.fc1 = nn.Linear(784, 200)
        self.fc2 = nn.Linear(200, 50)
        self.fc3 = nn.Linear(50, 10)

    def forward(self, x):
        x = x.view(x.shape[0],-1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))        
        x = F.relu(self.fc3(x))
        return F.log_softmax(x)
      


In [0]:
  #Return the samples that each client is going to have as a private training data set. This is a not overlapping set
  def get_samples(num_clients):
    tam = len(mnist_trainset)
    split= int(tam/num_clients)
    split_ini = split
    indices = list(range(tam))
    init=0
    samples = []
    for i in range(num_clients):     
      t_idx = indices[init:split]
      t_sampler = SubsetRandomSampler(t_idx)
      samples.append(t_sampler)
      init = split
      split = split+split_ini
    return samples 

In [0]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5, ), (0.5,))])
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
device =  torch.device("cuda:0""cuda:0" if torch.cuda.is_available() else "cpu")
num_clients = 100
train_len = len(mnist_trainset)
test_len = len(mnist_testset)

In [0]:
#We're creating the Server class. A priv_budget of 0.001 (the max delta) and a Epsilon of 8
serv = server(num_clients, 0.001, 8)
serv.server_exec(30)

Comunication round:  24
Accuracy:  0.8672
Delta spent:  0.001221426302927119
Delta budget:  0.001
